## Data Preprocessing


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import tensorflow as tf

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data') 

# Actions that we try to detect
actions = np.array(["HI","THANKS","FISH","COW","PLANE","CYCLING"])


no_sequences = 45


sequence_length = 20



In [3]:
label_map = {label:num for num, label in enumerate(actions)}

In [4]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [5]:
label_map


{'HI': 0, 'THANKS': 1, 'FISH': 2, 'COW': 3, 'PLANE': 4, 'CYCLING': 5}

In [6]:
np.array(sequences).shape

(270, 20, 258)

In [7]:
X = np.array(sequences)

In [8]:
y = to_categorical(labels).astype(int)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [93]:
y_test.shape


(81, 6)

In [94]:
(21*3)+(21*3)+(33*4)

258

## Model

In [95]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [99]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [100]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy', 
              metrics=['categorical_accuracy'])

In [101]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
6/6 [==============================] - 160s 1s/step - loss: 1.6807 - categorical_accuracy: 0.2910
Epoch 2/20
6/6 [==============================] - 7s 1s/step - loss: 1.2160 - categorical_accuracy: 0.5026
Epoch 3/20
6/6 [==============================] - 5s 895ms/step - loss: 0.9578 - categorical_accuracy: 0.5556
Epoch 4/20
6/6 [==============================] - 6s 1s/step - loss: 0.7760 - categorical_accuracy: 0.6614
Epoch 5/20
6/6 [==============================] - 7s 1s/step - loss: 0.5286 - categorical_accuracy: 0.7831
Epoch 6/20
6/6 [==============================] - 7s 766ms/step - loss: 0.3538 - categorical_accuracy: 0.8677
Epoch 7/20
6/6 [==============================] - 3s 528ms/step - loss: 0.3145 - categorical_accuracy: 0.9206
Epoch 8/20
6/6 [==============================] - 3s 550ms/step - loss: 0.1750 - categorical_accuracy: 0.9365
Epoch 9/20
6/6 [==============================] - 4s 624ms/step - loss: 0.3500 - categorical_accuracy: 0.8783
Epoch 10/20
6/6 [===

In [102]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 20, 64)            82688     
                                                                 
 lstm_22 (LSTM)              (None, 20, 128)           98816     
                                                                 
 lstm_23 (LSTM)              (None, 64)                49408     
                                                                 
 dense_21 (Dense)            (None, 64)                4160      
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 6)                 198       
                                                                 
Total params: 237,350
Trainable params: 237,350
Non-tr

In [103]:
res = model.predict(X_test)

3/3 [==============================] - 5s 125ms/step


In [111]:
model.save('isl6.h5')

In [104]:
yhat = model.predict(X_test)

3/3 [==============================] - 1s 135ms/step


In [105]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [106]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[67,  2],
        [ 0, 12]],

       [[68,  0],
        [ 0, 13]],

       [[69,  1],
        [ 0, 11]],

       [[64,  0],
        [ 0, 17]],

       [[67,  0],
        [ 2, 12]],

       [[67,  0],
        [ 1, 13]]], dtype=int64)

In [107]:
accuracy_score(ytrue, yhat)

0.9629629629629629

In [108]:
from sklearn.metrics import precision_score, recall_score
precision_score(ytrue, yhat,average='macro')

0.9623015873015873

In [109]:
recall_score(ytrue, yhat,average='macro')

0.9642857142857143

In [110]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss: {}, Test Accuracy: {}'.format(loss, accuracy))

3/3 [==============================] - 6s 156ms/step - loss: 0.1127 - categorical_accuracy: 0.9630
Test Loss: 0.11270470172166824, Test Accuracy: 0.9629629850387573
